# Importing Packages

In [1]:
from flask import Flask,  jsonify,request
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np
import pandas as pd
import string


from keras.layers import Input, Dense
from keras.models import Model
import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import LSTM

from nltk.tokenize import word_tokenize

from textblob import TextBlob

In [2]:
app = Flask(__name__)

# Uploading jobs_data csvfile

In [4]:
original_data_loaded = pd.read_csv('sample_jobs_data.csv') #original data loaded
original_data_loaded.columns = ['title', 'jobFunction', 'industry','skills','requirements','description'] # cloumns we have

enhanced_data=pd.read_csv('sample_jobs_data.csv') #enhanced data will be saved here
enhanced_data.columns = ['title', 'jobFunction', 'industry','skills','requirements','description'] # cloumns we have

enhanced_data.sample(10)# discover data we have

,title,jobFunction,industry,skills,requirements,description
191892,Graphic Designer,['Creative/Design/Art'],"['Information Technology Services', 'Marketing...","['Adobe Indesign', 'Advertising', 'Freehand Dr...","['Experience in Adobe Photoshop.', 'Minimum ex...",Design and finalize visuals and posters we use...
53602,Call Center Agent,['Customer Service/Support'],['Construction - Residential & Commercial/Offi...,"['Customer Service', 'Call Center', 'Customer ...",['Customer support experience or experience as...,Manage large amounts of incoming calls|Maintai...
136318,Purchasing & Warehouse Specialist - مشتريات مخازن,"['Logistics/Supply Chain', 'Purchasing/Procure...",['Marketing and Advertising'],"['Logistics', 'Inventory', 'Purchasing', 'Ware...","['Preferably self owned car.', 'English', 'Gen...",Responsible of Warehouse Inventory|Responsible...
121371,Math Co-Teacher,['Education/Teaching'],['Education'],"['Math', 'Education', 'Teaching']","['Fluency in English language.', 'Talent, crea...",Ensuring that all students succeed academicall...
167774,Export Sales Support Specialist,"['Engineering - Mechanical/Electrical', 'Sales...","['Electronics and Semiconductors', 'Engineerin...","['Sales', 'Sales Skills', 'Electrical Engineer...","['Good technical knowledge, preferably on ABB ...",Understanding of tenders with respect to scope...
143957,Customer Care Manager,['Customer Service/Support'],"['Manufacturing', 'Automotive']","['Customer Service', 'Customer Support', 'Cust...","['Problem solving skills', 'Time Management', ...",Supervise day to day operations in the custome...
55720,Senior Dot Net Developer,"['IT/Software Development', 'Engineering - Tel...",['Information Technology Services'],"['REST API', 'Software Development', 'C#', 'Co...","['Strong knowledge of Dot Net ecosystem, C# la...",Full life cycle application development.|Devel...
163161,Networks Engineer,"['IT/Software Development', 'Engineering - Tel...","['Computer Networking', 'Information Technolog...","['Installation', 'Troubleshooting', 'Computer ...","['Hands on experience in Aruba Technology', 'C...",1st level of Support & troubleshooting (For ru...
172512,Junior IT Support Engineer,"['IT/Software Development', 'Installation/Main...","['Information Technology Services', 'Business ...","['Technical Support', 'Linux', 'System Adminis...","['Military Status: Exempted, Done, Postpone', ...",Provide End-User with S/W and H/W Technical Su...
188423,Recruiter,['Human Resources'],"['Healthcare and Medical Services', 'Consultin...","['Microsoft Word', 'Interviewing', 'Screening'...","['Preferred HR Certificate', 'Age: maximum of ...",Provide support to an organization in finding ...


In [5]:
#removing special characters
enhanced_data['title']=enhanced_data['title'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data['jobFunction']=enhanced_data['jobFunction'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data['industry']=enhanced_data['industry'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data['skills']=enhanced_data['skills'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data['requirements']=enhanced_data['requirements'].str.encode('ascii', 'ignore').str.decode('ascii')
enhanced_data['description']=enhanced_data['description'].str.encode('ascii', 'ignore').str.decode('ascii')

# Dropping all rows that have missing value or 'nan' values

In [6]:
'''-------------------set Data Loaded in DataFrams--------------------'''
# old_original datafram that has the values without any enhancements
#old_original_df = pd.DataFrame(original_data_loaded)

# original datafram that has the values without any enhancements
#original_df = pd.DataFrame(original_data_loaded)

# old_enhanced datafram that has the values with enhancements
#old_enhanced_df = pd.DataFrame(enhanced_data)

# enhanced datafram that has the values with enhancements
#enhanced_df = pd.DataFrame(enhanced_data)

# enhanced datafram that has the values with enhancements
#play_df = pd.DataFrame(enhanced_data)

'-------------------set Data Loaded in DataFrams--------------------'

In [7]:
enhanced_data['string_description'] = ""

In [8]:
c=0

for item in enhanced_data['description']:
    z=isinstance(item, float)
    if(z == 1):
        enhanced_data['string_description'][c]=str(item)
    else:
        enhanced_data['string_description'][c]=item
    c=c+1

In [9]:
'''-----------------Get rows that have 'nan' values-------------------'''
# detect all the rows that contain 'nan' value from the enhanced enhanced_df
indexNames = enhanced_data[ (enhanced_data['title'] == "nan") | (enhanced_data['jobFunction'] == "['nan']") | (enhanced_data['industry'] == "['nan']")|(enhanced_data['skills'] == "['nan']") |(enhanced_data['requirements'] == "['nan']") |( enhanced_data['string_description'] == 'nan')].index

In [10]:
'''-----------------Drop Missing Values in Data Fram------------------'''
# drop all rows that has missing values from the data fram original_df
original_data_loaded.dropna(axis=0, how='any',inplace=False)

# drop all rows that has missing values from the data fram enhanced_df
enhanced_data.dropna(axis=0, how='any',inplace=False)

# drop all rows that has missing values from the data fram enhanced_df
#play_df.dropna(axis=0, how='any',inplace=False)

,title,jobFunction,industry,skills,requirements,description,string_description
0,Math Teacher,['Education/Teaching'],['Education'],"['Math', 'Education', 'Teaching']","['Creativity', 'Excellent communication skills...","Math teachers actively instruct students, crea...","Math teachers actively instruct students, crea..."
1,English Instructor - 6th Of October (Part Time),"['Training/Instructor', 'Education/Teaching']",['Education'],"['Translation', 'Linguistics', 'IQP', 'Educati...","['Great sense of responsibility, integrity and...",Deliver lessons in accordance with Berlitz tea...,Deliver lessons in accordance with Berlitz tea...
2,Java Technical Lead,"['IT/Software Development', 'Engineering - Tel...","['Information Technology Services', 'Computer ...","['Technical Advice', 'Software Engineering', '...",['Bachelor degree of Computer Science/ Compute...,Advises others and assumes a technical leaders...,Advises others and assumes a technical leaders...
3,KG Teacher Assistant,['Education/Teaching'],['Education'],"['Education', 'ICT', 'KG', 'Teaching']",['Excellent command of written & spoken Englis...,Contribute effectively to the teachers prepara...,Contribute effectively to the teachers prepara...
4,Accountant,['Accounting/Finance'],['Education'],"['School', 'Accounting', 'Billing', 'Finance',...","[""Bachelor's degree in accounting."", 'Presenta...",Prepare Documents such as monthly reports and ...,Prepare Documents such as monthly reports and ...
...,...,...,...,...,...,...,...
199995,International Property Consultant,['Sales/Retail'],"['Real Estate/Property Management', 'Business ...","['Sales', 'Real Estate', 'Property Consultant'...","['Excellent presentation skills', 'Education: ...",Achieve agreed sales targets and outcomes with...,Achieve agreed sales targets and outcomes with...
199996,"Applications Unlimited Sales Representative, L...","['IT/Software Development', 'Sales/Retail']","['Information Technology Services', 'Computer ...","['Peoplesoft', 'Sales', 'Computer Science', 'O...","['High level of energy, drive, enthusiasm, com...",Achieve individual revenue targets (indirect /...,Achieve individual revenue targets (indirect /...
199997,"Applications Unlimited Sales Representative, T...","['IT/Software Development', 'Sales/Retail']","['Information Technology Services', 'Computer ...","['Peoplesoft', 'Sales', 'Sales Skills', 'Compu...","['High level of energy, drive, enthusiasm, com...",Achieve individual revenue targets (indirect /...,Achieve individual revenue targets (indirect /...
199998,Sports Animator,"['Hospitality/Hotels/Food Services', 'Sports a...",['Real Estate/Property Management'],"['Sports', 'Entertainment', 'Customer Service'...","['Team Player', 'Must be physically fit', 'Bet...",Organize and run an entertainment program for ...,Organize and run an entertainment program for ...


In [11]:
'''-------------Delete Rows that contains 'nan' values----------------'''
# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the original_df
original_data_loaded.drop(indexNames , inplace=True)

# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the enhanced_df
enhanced_data.drop(indexNames , inplace=True)

# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the enhanced_df
#play_df.drop(indexNames , inplace=True)

In [12]:
'''------------------Reset Indexies in Data Fram----------------------'''
#reset the index of the original_df
original_data_loaded=original_data_loaded.reset_index(drop=True)

#reset the index of the enhanced_df
enhanced_data=enhanced_data.reset_index(drop=True)

#reset column 'id' in original_df
#original_data_loaded['id'] = original_data_loaded.index

#reset column 'id' in the enhanced_df
#enhanced_data['id'] = enhanced_data.index

#reset the index of the enhanced_df
#play_df=play_df.reset_index(drop=True)

#reset column 'id' in the enhanced_df
#play_df['id'] = play_df.index

# Pre processing

In [13]:
ps = PorterStemmer()
lemma = WordNetLemmatizer()
# excluding 'it' from the stopwords as it has meaning in the jobs the file have like 'IT/Software'
stop= set(stopwords.words('english')) - set(['it'])

In [14]:
#tokenization will done here too
#lower case all letters
enhanced_data['title'] = enhanced_data['title'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data['jobFunction'] = enhanced_data['jobFunction'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data['industry'] = enhanced_data['industry'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data['skills'] = enhanced_data['skills'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data['requirements'] = enhanced_data['requirements'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data['description'] = enhanced_data['description'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()
enhanced_data['string_description'] = enhanced_data['string_description'].str.lower().str.replace(r'[^\w\s]+', ' ').str.split()

In [15]:
enhanced_data

,title,jobFunction,industry,skills,requirements,description,string_description
0,"[math, teacher]","[education, teaching]",[education],"[math, education, teaching]","[creativity, excellent, communication, skills,...","[math, teachers, actively, instruct, students,...","[math, teachers, actively, instruct, students,..."
1,"[english, instructor, 6th, of, october, part, ...","[training, instructor, education, teaching]",[education],"[translation, linguistics, iqp, education, tra...","[great, sense, of, responsibility, integrity, ...","[deliver, lessons, in, accordance, with, berli...","[deliver, lessons, in, accordance, with, berli..."
2,"[java, technical, lead]","[it, software, development, engineering, telec...","[information, technology, services, computer, ...","[technical, advice, software, engineering, sof...","[bachelor, degree, of, computer, science, comp...","[advises, others, and, assumes, a, technical, ...","[advises, others, and, assumes, a, technical, ..."
3,"[kg, teacher, assistant]","[education, teaching]",[education],"[education, ict, kg, teaching]","[excellent, command, of, written, spoken, engl...","[contribute, effectively, to, the, teachers, p...","[contribute, effectively, to, the, teachers, p..."
4,[accountant],"[accounting, finance]",[education],"[school, accounting, billing, finance, financi...","[bachelor, s, degree, in, accounting, presenta...","[prepare, documents, such, as, monthly, report...","[prepare, documents, such, as, monthly, report..."
...,...,...,...,...,...,...,...
190240,"[international, property, consultant]","[sales, retail]","[real, estate, property, management, business,...","[sales, real, estate, property, consultant, sa...","[excellent, presentation, skills, education, e...","[achieve, agreed, sales, targets, and, outcome...","[achieve, agreed, sales, targets, and, outcome..."
190241,"[applications, unlimited, sales, representativ...","[it, software, development, sales, retail]","[information, technology, services, computer, ...","[peoplesoft, sales, computer, science, outdoor...","[high, level, of, energy, drive, enthusiasm, c...","[achieve, individual, revenue, targets, indire...","[achieve, individual, revenue, targets, indire..."
190242,"[applications, unlimited, sales, representativ...","[it, software, development, sales, retail]","[information, technology, services, computer, ...","[peoplesoft, sales, sales, skills, computer, s...","[high, level, of, energy, drive, enthusiasm, c...","[achieve, individual, revenue, targets, indire...","[achieve, individual, revenue, targets, indire..."
190243,"[sports, animator]","[hospitality, hotels, food, services, sports, ...","[real, estate, property, management]","[sports, entertainment, customer, service, spo...","[team, player, must, be, physically, fit, betw...","[organize, and, run, an, entertainment, progra...","[organize, and, run, an, entertainment, progra..."


In [16]:
#removing stop words
enhanced_data['title'] = enhanced_data['title'].apply(lambda x: [item for item in x if item not in stop])
enhanced_data['jobFunction'] = enhanced_data['jobFunction'].apply(lambda x: [item for item in x if item not in stop])
enhanced_data['industry'] = enhanced_data['industry'].apply(lambda x: [item for item in x if item not in stop])
enhanced_data['skills'] = enhanced_data['skills'].apply(lambda x: [item for item in x if item not in stop])
enhanced_data['requirements'] = enhanced_data['requirements'].apply(lambda x: [item for item in x if item not in stop])
#enhanced_data['description'] = enhanced_data['requirements'].apply(lambda x: [item for item in x if item not in stop])
enhanced_data['string_description'] = enhanced_data['string_description'].apply(lambda x: [item for item in x if item not in stop])

In [17]:
enhanced_data

,title,jobFunction,industry,skills,requirements,description,string_description
0,"[math, teacher]","[education, teaching]",[education],"[math, education, teaching]","[creativity, excellent, communication, skills,...","[math, teachers, actively, instruct, students,...","[math, teachers, actively, instruct, students,..."
1,"[english, instructor, 6th, october, part, time]","[training, instructor, education, teaching]",[education],"[translation, linguistics, iqp, education, tra...","[great, sense, responsibility, integrity, flex...","[deliver, lessons, in, accordance, with, berli...","[deliver, lessons, accordance, berlitz, teachi..."
2,"[java, technical, lead]","[it, software, development, engineering, telec...","[information, technology, services, computer, ...","[technical, advice, software, engineering, sof...","[bachelor, degree, computer, science, computer...","[advises, others, and, assumes, a, technical, ...","[advises, others, assumes, technical, leadersh..."
3,"[kg, teacher, assistant]","[education, teaching]",[education],"[education, ict, kg, teaching]","[excellent, command, written, spoken, english,...","[contribute, effectively, to, the, teachers, p...","[contribute, effectively, teachers, preparatio..."
4,[accountant],"[accounting, finance]",[education],"[school, accounting, billing, finance, financi...","[bachelor, degree, accounting, presentation, s...","[prepare, documents, such, as, monthly, report...","[prepare, documents, monthly, reports, financi..."
...,...,...,...,...,...,...,...
190240,"[international, property, consultant]","[sales, retail]","[real, estate, property, management, business,...","[sales, real, estate, property, consultant, sa...","[excellent, presentation, skills, education, e...","[achieve, agreed, sales, targets, and, outcome...","[achieve, agreed, sales, targets, outcomes, wi..."
190241,"[applications, unlimited, sales, representativ...","[it, software, development, sales, retail]","[information, technology, services, computer, ...","[peoplesoft, sales, computer, science, outdoor...","[high, level, energy, drive, enthusiasm, commi...","[achieve, individual, revenue, targets, indire...","[achieve, individual, revenue, targets, indire..."
190242,"[applications, unlimited, sales, representativ...","[it, software, development, sales, retail]","[information, technology, services, computer, ...","[peoplesoft, sales, sales, skills, computer, s...","[high, level, energy, drive, enthusiasm, commi...","[achieve, individual, revenue, targets, indire...","[achieve, individual, revenue, targets, indire..."
190243,"[sports, animator]","[hospitality, hotels, food, services, sports, ...","[real, estate, property, management]","[sports, entertainment, customer, service, spo...","[team, player, must, physically, fit, 18, 25, ...","[organize, and, run, an, entertainment, progra...","[organize, run, entertainment, program, guests..."


In [18]:
#lemmatization , return the base or dictionary from the words , which is know as lema
enhanced_data['title']= enhanced_data['title'].apply(lambda x : [lemma.lemmatize(y) for y in x])
enhanced_data['jobFunction']= enhanced_data['jobFunction'].apply(lambda x : [lemma.lemmatize(y) for y in x])
enhanced_data['industry']= enhanced_data['industry'].apply(lambda x : [lemma.lemmatize(y) for y in x])
enhanced_data['skills']= enhanced_data['skills'].apply(lambda x : [lemma.lemmatize(y) for y in x])
enhanced_data['requirements']= enhanced_data['requirements'].apply(lambda x : [lemma.lemmatize(y) for y in x])
#enhanced_data['description']= enhanced_data['description'].apply(lambda x : [lemma.lemmatize(y) for y in x])
enhanced_data['string_description']= enhanced_data['string_description'].apply(lambda x : [lemma.lemmatize(y) for y in x])

In [19]:
enhanced_data

,title,jobFunction,industry,skills,requirements,description,string_description
0,"[math, teacher]","[education, teaching]",[education],"[math, education, teaching]","[creativity, excellent, communication, skill, ...","[math, teachers, actively, instruct, students,...","[math, teacher, actively, instruct, student, c..."
1,"[english, instructor, 6th, october, part, time]","[training, instructor, education, teaching]",[education],"[translation, linguistics, iqp, education, tra...","[great, sense, responsibility, integrity, flex...","[deliver, lessons, in, accordance, with, berli...","[deliver, lesson, accordance, berlitz, teachin..."
2,"[java, technical, lead]","[it, software, development, engineering, telec...","[information, technology, service, computer, s...","[technical, advice, software, engineering, sof...","[bachelor, degree, computer, science, computer...","[advises, others, and, assumes, a, technical, ...","[advises, others, assumes, technical, leadersh..."
3,"[kg, teacher, assistant]","[education, teaching]",[education],"[education, ict, kg, teaching]","[excellent, command, written, spoken, english,...","[contribute, effectively, to, the, teachers, p...","[contribute, effectively, teacher, preparation..."
4,[accountant],"[accounting, finance]",[education],"[school, accounting, billing, finance, financi...","[bachelor, degree, accounting, presentation, s...","[prepare, documents, such, as, monthly, report...","[prepare, document, monthly, report, financial..."
...,...,...,...,...,...,...,...
190240,"[international, property, consultant]","[sale, retail]","[real, estate, property, management, business,...","[sale, real, estate, property, consultant, sal...","[excellent, presentation, skill, education, en...","[achieve, agreed, sales, targets, and, outcome...","[achieve, agreed, sale, target, outcome, withi..."
190241,"[application, unlimited, sale, representative,...","[it, software, development, sale, retail]","[information, technology, service, computer, s...","[peoplesoft, sale, computer, science, outdoor,...","[high, level, energy, drive, enthusiasm, commi...","[achieve, individual, revenue, targets, indire...","[achieve, individual, revenue, target, indirec..."
190242,"[application, unlimited, sale, representative,...","[it, software, development, sale, retail]","[information, technology, service, computer, s...","[peoplesoft, sale, sale, skill, computer, scie...","[high, level, energy, drive, enthusiasm, commi...","[achieve, individual, revenue, targets, indire...","[achieve, individual, revenue, target, indirec..."
190243,"[sport, animator]","[hospitality, hotel, food, service, sport, lei...","[real, estate, property, management]","[sport, entertainment, customer, service, spor...","[team, player, must, physically, fit, 18, 25, ...","[organize, and, run, an, entertainment, progra...","[organize, run, entertainment, program, guest,..."


In [20]:
#Stemming , return the roots of the words and replacing the suffix, which is know as stem
#enhanced_data['title']= enhanced_data['title'].apply(lambda x : [ps.stem(y) for y in x])
#enhanced_data['jobFunction']= enhanced_data['jobFunction'].apply(lambda x : [ps.stem(y) for y in x])
#enhanced_data['industry']= enhanced_data['industry'].apply(lambda x : [ps.stem(y) for y in x])
#enhanced_data['skills']= enhanced_data['skills'].apply(lambda x : [ps.stem(y) for y in x])
#enhanced_data['requirements']= enhanced_data['requirements'].apply(lambda x : [ps.stem(y) for y in x])
#enhanced_data['description']= enhanced_data['description'].apply(lambda x : [ps.stem(y) for y in x])
#enhanced_data['string_description']= enhanced_data['string_description'].apply(lambda x : [ps.stem(y) for y in x])

In [21]:
#concatinate again all the values in each row into one string as [ux, designer] to 'ux designer'
# each row in 'title' column
counter1=0
for item in enhanced_data['title']:  
    s=""
    for i in item: 
        if i != 'nan':
            s=s+" "+i
        else:
            s=i
    enhanced_data['title'][counter1]=""
    enhanced_data['title'][counter1]=s
    counter1=counter1+1 

In [22]:
# each row in 'jobFunction' column
counter2=0
for item in enhanced_data['jobFunction']:  
    s=""
    for i in item: 
        if i != 'nan':
            s=s+" "+i
        else:
            s=i
    enhanced_data['jobFunction'][counter2]=""
    enhanced_data['jobFunction'][counter2]=s
    counter2=counter2+1

In [23]:
# each row in 'industry' column
counter3=0
for item in enhanced_data['industry']:  
    s=""
    for i in item: 
        if i != 'nan':
            s=s+" "+i
        else:
            s=i
    enhanced_data['industry'][counter3]=""
    enhanced_data['industry'][counter3]=s
    counter3=counter3+1

In [24]:
# each row in 'skills' column
counter4=0
for item in enhanced_data['skills']:  
    s=""
    for i in item: 
        if i != 'nan':
            s=s+" "+i
        else:
            s=i
    enhanced_data['skills'][counter4]=""
    enhanced_data['skills'][counter4]=s
    counter4=counter4+1

In [25]:
# each row in 'requirements' column
counter5=0
for item in enhanced_data['requirements']:  
    s=""
    for i in item: 
        if i != 'nan':
            s=s+" "+i
        else:
            s=i
    enhanced_data['requirements'][counter5]=""
    enhanced_data['requirements'][counter5]=s
    counter5=counter5+1

In [26]:
# each row in 'string_description' column
counter6=0
for item in enhanced_data['string_description']:  
    s=""
    for i in item: 
        if i != 'nan':
            s=s+" "+i
        else:
            s=i
    enhanced_data['string_description'][counter6]=""
    enhanced_data['string_description'][counter6]=s
    counter6=counter6+1

In [27]:
enhanced_data

,title,jobFunction,industry,skills,requirements,description,string_description
0,math teacher,education teaching,education,math education teaching,creativity excellent communication skill look...,"[math, teachers, actively, instruct, students,...",math teacher actively instruct student create...
1,english instructor 6th october part time,training instructor education teaching,education,translation linguistics iqp education trainin...,great sense responsibility integrity flexibil...,"[deliver, lessons, in, accordance, with, berli...",deliver lesson accordance berlitz teaching pr...
2,java technical lead,it software development engineering telecom t...,information technology service computer software,technical advice software engineering softwar...,bachelor degree computer science computer eng...,"[advises, others, and, assumes, a, technical, ...",advises others assumes technical leadership r...
3,kg teacher assistant,education teaching,education,education ict kg teaching,excellent command written spoken english time...,"[contribute, effectively, to, the, teachers, p...",contribute effectively teacher preparation le...
4,accountant,accounting finance,education,school accounting billing finance financial s...,bachelor degree accounting presentation skill...,"[prepare, documents, such, as, monthly, report...",prepare document monthly report financial sta...
...,...,...,...,...,...,...,...
190240,international property consultant,sale retail,real estate property management business serv...,sale real estate property consultant sale tar...,excellent presentation skill education englis...,"[achieve, agreed, sales, targets, and, outcome...",achieve agreed sale target outcome within sch...
190241,application unlimited sale representative levant,it software development sale retail,information technology service computer software,peoplesoft sale computer science outdoor sale...,high level energy drive enthusiasm commitment...,"[achieve, individual, revenue, targets, indire...",achieve individual revenue target indirect di...
190242,application unlimited sale representative turkey,it software development sale retail,information technology service computer software,peoplesoft sale sale skill computer science i...,high level energy drive enthusiasm commitment...,"[achieve, individual, revenue, targets, indire...",achieve individual revenue target indirect di...
190243,sport animator,hospitality hotel food service sport leisure,real estate property management,sport entertainment customer service sport an...,team player must physically fit 18 25 year ol...,"[organize, and, run, an, entertainment, progra...",organize run entertainment program guest resp...


In [28]:
# save the enhanced_data in enhanced_data.csv file,so we don't have to reclean the data each time we want to use it again 
#enhanced_data.to_csv('sample_jobs_skills_enhanced_data2.csv')
#upload enhanced_data.csv
#enhanced_data = pd.read_csv('enhanced_data.csv')

In [11]:
#upload enhanced_data.csv
enhanced_data = pd.read_csv('sample_jobs_skills_enhanced_data2.csv')

In [12]:
enhanced_data

,Unnamed: 0,title,jobFunction,industry,skills,requirements,description,string_description
0,0,math teacher,education teaching,education,math education teaching,creativity excellent communication skill look...,"['math', 'teachers', 'actively', 'instruct', '...",math teacher actively instruct student create...
1,1,english instructor 6th october part time,training instructor education teaching,education,translation linguistics iqp education trainin...,great sense responsibility integrity flexibil...,"['deliver', 'lessons', 'in', 'accordance', 'wi...",deliver lesson accordance berlitz teaching pr...
2,2,java technical lead,it software development engineering telecom t...,information technology service computer software,technical advice software engineering softwar...,bachelor degree computer science computer eng...,"['advises', 'others', 'and', 'assumes', 'a', '...",advises others assumes technical leadership r...
3,3,kg teacher assistant,education teaching,education,education ict kg teaching,excellent command written spoken english time...,"['contribute', 'effectively', 'to', 'the', 'te...",contribute effectively teacher preparation le...
4,4,accountant,accounting finance,education,school accounting billing finance financial s...,bachelor degree accounting presentation skill...,"['prepare', 'documents', 'such', 'as', 'monthl...",prepare document monthly report financial sta...
...,...,...,...,...,...,...,...,...
190240,190240,international property consultant,sale retail,real estate property management business serv...,sale real estate property consultant sale tar...,excellent presentation skill education englis...,"['achieve', 'agreed', 'sales', 'targets', 'and...",achieve agreed sale target outcome within sch...
190241,190241,application unlimited sale representative levant,it software development sale retail,information technology service computer software,peoplesoft sale computer science outdoor sale...,high level energy drive enthusiasm commitment...,"['achieve', 'individual', 'revenue', 'targets'...",achieve individual revenue target indirect di...
190242,190242,application unlimited sale representative turkey,it software development sale retail,information technology service computer software,peoplesoft sale sale skill computer science i...,high level energy drive enthusiasm commitment...,"['achieve', 'individual', 'revenue', 'targets'...",achieve individual revenue target indirect di...
190243,190243,sport animator,hospitality hotel food service sport leisure,real estate property management,sport entertainment customer service sport an...,team player must physically fit 18 25 year ol...,"['organize', 'and', 'run', 'an', 'entertainmen...",organize run entertainment program guest resp...


# Now we have our Data Loaded Tokenized,  Lemmatizied, Lowercased, no Special Characters no Missing Values, no 'nan' Values

### Now we are ready to use our data...

# 1-Extract Noun phrases using TextBlob

In [13]:
#!python -m textblob.download_corpora

In [15]:
enhanced_data['noun phrases'] = ""

In [16]:
c=0
for each_row in enhanced_data['string_description']:
    blob = TextBlob(str(each_row))
    phrases_list=list()
    for nouns in blob.noun_phrases:
        phrases_list.append(nouns)
    enhanced_data['noun phrases'][c]=phrases_list
    c=c+1

G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [17]:
enhanced_data

,Unnamed: 0,title,jobFunction,industry,skills,requirements,description,string_description,noun phrases
0,0,math teacher,education teaching,education,math education teaching,creativity excellent communication skill look...,"['math', 'teachers', 'actively', 'instruct', '...",math teacher actively instruct student create...,"[math teacher, instruct student, lesson plan, ..."
1,1,english instructor 6th october part time,training instructor education teaching,education,translation linguistics iqp education trainin...,great sense responsibility integrity flexibil...,"['deliver', 'lessons', 'in', 'accordance', 'wi...",deliver lesson accordance berlitz teaching pr...,"[lesson accordance berlitz teaching principle,..."
2,2,java technical lead,it software development engineering telecom t...,information technology service computer software,technical advice software engineering softwar...,bachelor degree computer science computer eng...,"['advises', 'others', 'and', 'assumes', 'a', '...",advises others assumes technical leadership r...,"[advises others assumes, technical leadership ..."
3,3,kg teacher assistant,education teaching,education,education ict kg teaching,excellent command written spoken english time...,"['contribute', 'effectively', 'to', 'the', 'te...",contribute effectively teacher preparation le...,"[teacher preparation lesson, setting learning ..."
4,4,accountant,accounting finance,education,school accounting billing finance financial s...,bachelor degree accounting presentation skill...,"['prepare', 'documents', 'such', 'as', 'monthl...",prepare document monthly report financial sta...,"[monthly report, financial statement, analyze ..."
...,...,...,...,...,...,...,...,...,...
190240,190240,international property consultant,sale retail,real estate property management business serv...,sale real estate property consultant sale tar...,excellent presentation skill education englis...,"['achieve', 'agreed', 'sales', 'targets', 'and...",achieve agreed sale target outcome within sch...,"[sale target outcome, international sale proce..."
190241,190241,application unlimited sale representative levant,it software development sale retail,information technology service computer software,peoplesoft sale computer science outdoor sale...,high level energy drive enthusiasm commitment...,"['achieve', 'individual', 'revenue', 'targets'...",achieve individual revenue target indirect di...,"[individual revenue target, territory pipeline..."
190242,190242,application unlimited sale representative turkey,it software development sale retail,information technology service computer software,peoplesoft sale sale skill computer science i...,high level energy drive enthusiasm commitment...,"['achieve', 'individual', 'revenue', 'targets'...",achieve individual revenue target indirect di...,"[individual revenue target, territory pipeline..."
190243,190243,sport animator,hospitality hotel food service sport leisure,real estate property management,sport entertainment customer service sport an...,team player must physically fit 18 25 year ol...,"['organize', 'and', 'run', 'an', 'entertainmen...",organize run entertainment program guest resp...,"[run entertainment program guest, physical act..."


In [18]:
# save the enhanced_data in enhanced_data.csv file,so we don't have to reclean the data each time we want to use it again 
#enhanced_data.to_csv('sample_jobs_skills_enhanced_data_with_noun_phrases2.csv')


In [19]:
#upload enhanced_data.csv
#enhanced_data = pd.read_csv('sample_jobs_skills_enhanced_data_with_noun_phrases1.csv')

# Creating a dataframe for all noun phrases , its context, and its description sentence from our data (a sample of 1000 record are just used to train the model)

In [22]:
# enhanced datafram that has the values with enhancements
dataframe = pd.DataFrame()

In [23]:
dataframe['noun_phrase']=""
dataframe['context']=""
dataframe['sentence']=""

In [24]:
count=0

for each_row in enhanced_data["string_description"]:
    if (count==1000):
        break
    else:
        s=""
        s=each_row

        for each_np_elemnt in enhanced_data["noun phrases"][count]:

            new_String_np=""
            new_String_np=new_String_np+each_np_elemnt

            large_list = s.partition(each_np_elemnt)

            tokinized_before_noun_phrase = word_tokenize(str(large_list[0]))
            tokinized_after_noun_phrase = word_tokenize(str(large_list[2]))

            context=list()

            #take context before noun phrase (3 words before)
            if((len(tokinized_before_noun_phrase)!=0) & (len(tokinized_before_noun_phrase)<=3)):
                for i in tokinized_before_noun_phrase:
                    context.append(i)


            elif ((len(tokinized_before_noun_phrase)!=0) & (len(tokinized_before_noun_phrase)>=3)):
                context.append(tokinized_before_noun_phrase[-3])
                context.append(tokinized_before_noun_phrase[-2])
                context.append(tokinized_before_noun_phrase[-1])

            #take context after noun phrase (3 words after)
            if((len(tokinized_after_noun_phrase)!=0) & (len(tokinized_after_noun_phrase)<=3)):
                for i in tokinized_after_noun_phrase:
                    context.append(i)  

            elif ((len(tokinized_after_noun_phrase)!=0) & (len(tokinized_after_noun_phrase)>=3)):
                context.append(tokinized_after_noun_phrase[0])
                context.append(tokinized_after_noun_phrase[1])
                context.append(tokinized_after_noun_phrase[2])   

            data = [{'noun_phrase':new_String_np ,'context':context,'sentence':s}]

            dataframe=dataframe.append(data , ignore_index=True , sort=False)
        count=count+1

In [25]:
dataframe

,noun_phrase,context,sentence
0,math teacher,"[actively, instruct, student]",math teacher actively instruct student create...
1,instruct student,"[math, teacher, actively, create, lesson, plan]",math teacher actively instruct student create...
2,lesson plan,"[instruct, student, create, assign, correct, h...",math teacher actively instruct student create...
3,homework manage student classroom communicate ...,"[plan, assign, correct, help, student, prepare]",math teacher actively instruct student create...
4,lesson accordance berlitz teaching principle,"[deliver, focused, interest, need]",deliver lesson accordance berlitz teaching pr...
...,...,...,...
12448,technical specification,"[written, business, requirement, document, per...",develop application desktop web platform writ...
12449,analyze document,"[technical, specification, investigate, report...",develop application desktop web platform writ...
12450,unit test plan script test harness,"[create, document, implement, create, maintain...",develop application desktop web platform writ...
12451,technical documentation,"[harness, create, maintain, using, defined, te...",develop application desktop web platform writ...


# Build Training Dataset (skill and not skill dataset)

In [259]:
all_jobtitles =""
for e in enhanced_data['title']:
    all_jobtitles=all_jobtitles+str(e)
    all_jobtitles=all_jobtitles+" "
    
resultString_jobtitles=remov_duplicates(all_jobtitles)

all_jobfunctions =""
for e in enhanced_data['jobFunction']:
    all_jobfunctions=all_jobfunctions+str(e)
    all_jobfunctions=all_jobfunctions+" "
    
resultString_jobfunctions=remov_duplicates(all_jobfunctions)

all_industries =""
for e in enhanced_data['industry']:
    all_industries=all_industries+str(e)
    all_industries=all_industries+" "
    
resultString_industries=remov_duplicates(all_industries)

all_skills =""
for e in enhanced_data['skills']:
    all_skills=all_skills+str(e)
    all_skills=all_skills+" "
    
resultString_skills=remov_duplicates(all_skills)

In [258]:
from collections import Counter 
  
def remov_duplicates(input): 
  
    # split input string separated by space 
    input = input.split(" ") 
  
    # joins two adjacent elements in iterable way 
    for i in range(0, len(input)): 
        input[i] = "".join(input[i]) 
  
    # now create dictionary using counter method 
    # which will have strings as key and their  
    # frequencies as value 
    UniqW = Counter(input) 
  
    # joins two adjacent elements in iterable way 
    s = " ".join(UniqW.keys()) 
    return s

In [260]:
#Insert data in a data fram

# Calling DataFrame constructor 
new_df = pd.DataFrame() 
new_df["input"] = ""
new_df["skill_or_not"] = ""

data1 = resultString_jobtitles.split(" ") #split string into a list
data2 = resultString_jobfunctions.split(" ") #split string into a list
data3 = resultString_industries.split(" ") #split string into a list
data4 = resultString_skills.split(" ") #split string into a list

for temp in data1:
    new_df = new_df.append({'input': temp, 'skill_or_not': 0}, ignore_index=True)
    
for temp in data2:
    new_df = new_df.append({'input': temp, 'skill_or_not': 0}, ignore_index=True)
    
for temp in data3:
    new_df = new_df.append({'input': temp, 'skill_or_not': 0}, ignore_index=True)
    
for temp in data4:
    new_df = new_df.append({'input': temp, 'skill_or_not': 1}, ignore_index=True)

In [261]:
#Shuffle Training Dataset Rows

#print("Dataframe Contens ", new_df, sep='\n')
from sklearn.utils import shuffle
new_df = shuffle(new_df)
new_df.reset_index(drop=True)

,input,skill_or_not
0,staffing,1
1,simulink,1
2,corporate,1
3,epm,0
4,maintenance,1
...,...,...
4990,nginx,1
4991,jr,0
4992,forensics,1
4993,prourement,1


In [372]:
# Save the skill_or_not_skill Dataset into csv file for further use in skills_not_skills1.csv file

#new_df.to_csv('skills_not_skills1.csv')

# Embedding skills we have by google universal encoder. 

In [194]:
#Function so that one session can be called multiple times. 
#Useful while multiple calls need to be done for embedding. 
import tensorflow as tf
import tensorflow_hub as hub

In [24]:
#download the model to local so it can be used again and again
#!mkdir G:\sentence_wise_email\module\module_useT

In [25]:
# Download the module, and uncompress it to the destination folder. 
#!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC G:\sentence_wise_email\module\module_useT

In [195]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

In [197]:
embed_fn = embed_useT('G:\sentence_wise_email\module\module_useT')

In [209]:
#upload enhanced_data.csv
new_df = pd.read_csv('skills_not_skills1.csv')

In [221]:
'''-----------------Get rows that have '0' values-------------------'''
# detect all the rows that contain '0' value
indexNames = new_df[ (new_df['skill_or_not'] == 0)].index

In [222]:
'''-------------Delete Rows that contains 'nan' values----------------'''
# delete all rows for which column 'title','jobFunction','industry' has value'nan' from the original_df
new_df.drop(indexNames , inplace=True)

In [236]:
skills=list()

In [238]:
for u in new_df["input"]:
    skills.append(str(u))

In [239]:
skill_matrix = embed_fn(skills)

# Predict whether each noun_phrase we have is a skill or not a skill, by  getting similarity between any new input_noun_phrase and the skills dataset we create, if the similarity is greater than 0.7 , so the input_noun_phrase is a skill >>>>>>>>>>>>>>>>>

In [247]:
dataframe["skill_or_not"]=""

In [249]:
c=0
for test_text in dataframe['noun_phrase']:
    
    test_embed = embed_fn([test_text])
    np.inner(test_embed, skill_matrix)
    
    sim_matrix  = np.inner(test_embed, skill_matrix)
    
    if sim_matrix.max() > 0.7:
        dataframe["skill_or_not"][c]=1
    else:
        dataframe["skill_or_not"][c]=0
        
    c=c+1

In [250]:
dataframe

,noun_phrase,context,sentence,skill_or_not
0,math teacher,"[actively, instruct, student]",math teacher actively instruct student create...,1
1,instruct student,"[math, teacher, actively, create, lesson, plan]",math teacher actively instruct student create...,1
2,lesson plan,"[instruct, student, create, assign, correct, h...",math teacher actively instruct student create...,1
3,homework manage student classroom communicate ...,"[plan, assign, correct, help, student, prepare]",math teacher actively instruct student create...,1
4,lesson accordance berlitz teaching principle,"[deliver, focused, interest, need]",deliver lesson accordance berlitz teaching pr...,1
...,...,...,...,...
12448,technical specification,"[written, business, requirement, document, per...",develop application desktop web platform writ...,0
12449,analyze document,"[technical, specification, investigate, report...",develop application desktop web platform writ...,1
12450,unit test plan script test harness,"[create, document, implement, create, maintain...",develop application desktop web platform writ...,0
12451,technical documentation,"[harness, create, maintain, using, defined, te...",develop application desktop web platform writ...,1


# Create Training dataset that we will pass to LSTM Model

In [275]:
#training dataset
# enhanced datafram that has the values with enhancements
new_dataframe = pd.DataFrame()
new_dataframe['noun_phrase']=""
new_dataframe['context']=""
new_dataframe['sentence']=""
new_dataframe=dataframe .head(1000)  

# Create Test dataset that we will Evaluate LSTM Model with

In [321]:
# enhanced datafram that has the values with enhancements
test_dataframe = pd.DataFrame()
test_dataframe['noun_phrase']=""
test_dataframe['context']=""
test_dataframe['sentence']=""
test_dataframe=dataframe .tail(1001)

In [322]:
test_dataframe=test_dataframe[test_dataframe.astype(str)['context'] != '[]']

# Embedding Training Dataset by google universal encoder

In [32]:
#Noun phrases embedding-----------------------------------------------------------------------------
encoding_matrix_nounphrases = embed_fn(new_dataframe["noun_phrase"])
encoding_matrix_nounphrases.shape

#sum context words vectors , then get context embedding---------------------------------------------
large_list_context=list()
for each_list in new_dataframe['context']:
    vector=sum(embed_fn(each_list))
    large_list_context.append(vector)
    
encoding_matrix_context= np.array(large_list_context)
encoding_matrix_context.shape

#Description Sentence embedding-----------------------------------------------------------------------
encoding_matrix_sentences = embed_fn(new_dataframe["sentence"])
encoding_matrix_sentences.shape

# Embedding Test Dataset  by google universal encoder

In [324]:
#Noun phrases embedding-------------------------------------------------------------------------------
encoding_matrix_nounphrases_test = embed_fn(test_dataframe["noun_phrase"])
encoding_matrix_nounphrases_test.shape

#sum context words vectors , then get context embedding-----------------------------------------------
large_list_context_test = list()
for each_list in test_dataframe['context']:
    vector=sum(embed_fn(each_list))
    large_list_context_test.append(vector)
    
encoding_matrix_context_test = np.array(large_list_context_test)
encoding_matrix_context_test.shape

#Description Sentence embedding-----------------------------------------------------------------------
encoding_matrix_sentences_test = embed_fn(test_dataframe["sentence"])
encoding_matrix_sentences_test.shape

# Build network architecture

In [375]:
class SkillsExtractorNN:

    def __init__(self, word_features_dim, dense_features_dim):

        lstm_input_phrase = keras.layers.Input(shape=(None, word_features_dim))
        lstm_input_cont = keras.layers.Input(shape=(None, word_features_dim))
        dense_input = keras.layers.Input(shape=(None,dense_features_dim))

        lstm_emb_phrase = keras.layers.LSTM(256)(lstm_input_phrase)
        lstm_emb_phrase = keras.layers.Dense(128, activation='relu')(lstm_emb_phrase)

        lstm_emb_cont = keras.layers.LSTM(256)(lstm_input_cont)
        lstm_emb_cont = keras.layers.Dense(128, activation='relu')(lstm_emb_cont)
        
        lstm_emb_cont_new = keras.layers.LSTM(512)(dense_input)
        dense_emb = keras.layers.Dense(512, activation='relu')(lstm_emb_cont_new)
        dense_emb = keras.layers.Dense(256, activation='relu')(dense_emb)

        x = keras.layers.concatenate([lstm_emb_phrase, lstm_emb_cont, dense_emb])
        x = keras.layers.Dense(128, activation='relu')(x)
        x = keras.layers.Dense(64, activation='relu')(x)
        x = keras.layers.Dense(32, activation='relu')(x)

        main_output = keras.layers.Dense(2, activation='softplus')(x)

        self.model = keras.models.Model(inputs=[lstm_input_phrase, lstm_input_cont, dense_input],
                                        outputs=main_output)

        optimizer = keras.optimizers.Adam(lr=0.0001)

        self.model.compile(optimizer=optimizer, loss='binary_crossentropy')
        
        # summarize layers
        print(self.model.summary())
        
    def fit(self, x_lstm_phrase, x_lstm_context, x_dense, y,
            val_split=0.25, patience=5, max_epochs=1000, batch_size=32):
        
        
        data111 = encoding_matrix_nounphrases.reshape((encoding_matrix_nounphrases.shape[0],1, encoding_matrix_nounphrases.shape[1]))
        data222 = encoding_matrix_context.reshape((encoding_matrix_context.shape[0],1, encoding_matrix_context.shape[1]))
        data333 = encoding_matrix_sentences.reshape((encoding_matrix_sentences.shape[0],1, encoding_matrix_sentences.shape[1]))

        x_lstm_phrase_seq = keras.preprocessing.sequence.pad_sequences(data111)
        x_lstm_context_seq = keras.preprocessing.sequence.pad_sequences(data222)
        x_dense=data333

        y_onehot = onehot_transform(y)

        self.model.fit([x_lstm_phrase_seq, x_lstm_context_seq, x_dense],
                       y_onehot,
                       batch_size=batch_size,
                       epochs=max_epochs,
                       validation_split=val_split,
                       callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)])


    def predict(self, x_lstm_phrase, x_lstm_context, x_dense):
        
        reshaped_noun_phrases = encoding_matrix_nounphrases.reshape((x_lstm_phrase.shape[0],1, x_lstm_phrase.shape[1]))
        reshaped_context = encoding_matrix_context.reshape((x_lstm_context.shape[0],1, x_lstm_context.shape[1]))
        reshaped_sentences = encoding_matrix_sentences.reshape((x_dense.shape[0],1, x_dense.shape[1]))

        
        x_lstm_phrase_seq = keras.preprocessing.sequence.pad_sequences(reshaped_noun_phrases)
        x_lstm_context_seq = keras.preprocessing.sequence.pad_sequences(reshaped_context)

        y = self.model.predict([x_lstm_phrase_seq, x_lstm_context_seq, reshaped_sentences])

        return y

In [114]:
def onehot_transform(y):

    onehot_y = []

    for numb in y:
        onehot_arr = np.zeros(2)
        onehot_arr[numb] = 1
        onehot_y.append(np.array(onehot_arr))

    return np.array(onehot_y)

In [388]:
def return_one_hot_encoding_back(y):
    out_puts_hot_encoded=list()
    for i in y:

        small_list=list()

        if (i[0] >= 0.5) :
            var1=1      
        else:
            var1=0

        if (i[1] >=0.5):
            var2=1
        else:
            var2=0

        small_list.append(var1)
        small_list.append(var2)
        out_puts_hot_encoded.append(small_list)
        
        
    c=0
    out_puts_=list()
    for i in out_puts_hot_encoded:
        if ((i[0]==1) &(i[1]==0)):
            out_puts_.append(0)
        elif ((i[0]==0)&(i[1]==1)):
            out_puts_.append(1)
        
    return out_puts_

In [384]:
obj = SkillsExtractorNN(512,512)

Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_200 (InputLayer)          (None, None, 512)    0                                            
__________________________________________________________________________________________________
input_198 (InputLayer)          (None, None, 512)    0                                            
__________________________________________________________________________________________________
input_199 (InputLayer)          (None, None, 512)    0                                            
__________________________________________________________________________________________________
lstm_135 (LSTM)                 (None, 512)          2099200     input_200[0][0]                  
___________________________________________________________________________________________

# Start Trainining

In [385]:
train=obj.fit(encoding_matrix_nounphrases,encoding_matrix_context,encoding_matrix_sentences,new_dataframe['skill_or_not'])

Train on 750 samples, validate on 250 samples
Epoch 1/1000
750/750 [==============================] - 13s 17ms/step - loss: 0.7593 - val_loss: 0.7423
Epoch 2/1000
750/750 [==============================] - 4s 5ms/step - loss: 0.7090 - val_loss: 0.7045
Epoch 3/1000
750/750 [==============================] - 4s 5ms/step - loss: 0.6862 - val_loss: 0.7035
Epoch 4/1000
750/750 [==============================] - 3s 5ms/step - loss: 0.6851 - val_loss: 0.7018
Epoch 5/1000
750/750 [==============================] - 3s 5ms/step - loss: 0.6849 - val_loss: 0.7037
Epoch 6/1000
750/750 [==============================] - 3s 4ms/step - loss: 0.6846 - val_loss: 0.7039
Epoch 7/1000
750/750 [==============================] - 4s 5ms/step - loss: 0.6839 - val_loss: 0.7012
Epoch 8/1000
750/750 [==============================] - 3s 4ms/step - loss: 0.6807 - val_loss: 0.7040
Epoch 9/1000
750/750 [==============================] - 3s 5ms/step - loss: 0.6791 - val_loss: 0.7006
Epoch 10/1000
750/750 [===========

# Makining Predictions

In [386]:
y=obj.predict(encoding_matrix_nounphrases_test,encoding_matrix_context_test,encoding_matrix_sentences_test)

In [387]:
y.shape

(1000, 2)

In [389]:
test_dataframe['predicted_output']=0
'''------------------Reset Indexies in Data Fram----------------------'''
#reset the index of the original_df
test_dataframe=test_dataframe.reset_index(drop=True)

In [390]:
test_dataframe

,noun_phrase,context,sentence,skill_or_not,predicted_output
0,custom script increase system efficiency,"[strategy, write, maintain, lower, human, inte...",manage monitor installed system infrastructur...,0,0
1,human intervention time task,"[system, efficiency, lower, participate, desig...",manage monitor installed system infrastructur...,1,0
2,design information operational support system ...,"[time, task, participate, take, responsibility...",manage monitor installed system infrastructur...,1,0
3,responsibility configuration management linux ...,"[system, configuration, take, maintain, operat...",manage monitor installed system infrastructur...,1,0
4,additional software configure manage system ap...,"[maintain, operating, system, monitoring, cert...",manage monitor installed system infrastructur...,1,0
...,...,...,...,...,...
995,technical specification,"[written, business, requirement, document, per...",develop application desktop web platform writ...,0,0
996,analyze document,"[technical, specification, investigate, report...",develop application desktop web platform writ...,1,0
997,unit test plan script test harness,"[create, document, implement, create, maintain...",develop application desktop web platform writ...,0,0
998,technical documentation,"[harness, create, maintain, using, defined, te...",develop application desktop web platform writ...,1,0


In [391]:
out_puts_=return_one_hot_encoding_back(y)

In [392]:
c=0
for i in out_puts_:
    
    if (i==1):
        test_dataframe['predicted_output'][c]=1
        
    elif (i==0):
        test_dataframe['predicted_output'][c]=0
        
    c=c+1

G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
G:\AnacondaSetup\envs\New_Env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [393]:
test_dataframe

,noun_phrase,context,sentence,skill_or_not,predicted_output
0,custom script increase system efficiency,"[strategy, write, maintain, lower, human, inte...",manage monitor installed system infrastructur...,0,1
1,human intervention time task,"[system, efficiency, lower, participate, desig...",manage monitor installed system infrastructur...,1,1
2,design information operational support system ...,"[time, task, participate, take, responsibility...",manage monitor installed system infrastructur...,1,1
3,responsibility configuration management linux ...,"[system, configuration, take, maintain, operat...",manage monitor installed system infrastructur...,1,1
4,additional software configure manage system ap...,"[maintain, operating, system, monitoring, cert...",manage monitor installed system infrastructur...,1,1
...,...,...,...,...,...
995,technical specification,"[written, business, requirement, document, per...",develop application desktop web platform writ...,0,0
996,analyze document,"[technical, specification, investigate, report...",develop application desktop web platform writ...,1,0
997,unit test plan script test harness,"[create, document, implement, create, maintain...",develop application desktop web platform writ...,0,0
998,technical documentation,"[harness, create, maintain, using, defined, te...",develop application desktop web platform writ...,1,0


# Evaluate Our Model

In [394]:
''' Get the error score   --------------'''
#root mean square error of our model
mse = mean_squared_error(test_dataframe["skill_or_not"], test_dataframe["predicted_output"])
rmse = sqrt(mse)
print('RMSE: %f' % rmse) #error
print("")

RMSE: 0.692098



In [395]:
''' Get the Accuracy of our model ----------------'''

# Calculate accuracy percentage between two lists
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

accuracy = accuracy_metric(test_dataframe["skill_or_not"], test_dataframe["predicted_output"])
print('Accuracy: %f' % accuracy) #accuracy

Accuracy: 52.100000
